![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F1327132%2F0c83aef015cef72a1c6ef0bdb18937f3%2Fcolumns%20meaning%20small.png?generation=1609671531255584&alt=media)

In [ ]:
# basic

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# display

from IPython.display import display
from colorama import Fore
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load data

data=pd.read_csv('../input/marketing-data/marketing_data.csv')

# Section 01: Exploratory Data Analysis

### question

1. Are there any null values or outliers? How will you wrangle/handle them?
2. Are there any variables that warrant transformations?  
3. Are there any useful variables that you can engineer with the given data?
4. Do you notice any patterns or anomalies in the data? Can you plot them?

### answer

1. income 24 missing. and year_birth outliers. income fill nan to median and outlier slice.  
2. income remove percuation and change to float. also, dt_customer date format change. and like acceptedcmp, seems number but category. but put it now for the time being. because of calculate convenience. 
3. num_ , mnt_, cmp_ intergrated total. for remarkable insights.  
4. yes.    


* in products, wine meat exceptionally high. 
* i don't know marketing in details, but most of campaigns seems not-effective. 
* loyal customer ids : 1991, 7348, 5371, 4073 ... with wordcloud.

#### take eyes on data infos

Income - missing 24. and have punctuation.   
Dt_Customer - date format  

In [ ]:
def datainfo(df):
    return pd.DataFrame([(col,df[col].nunique(),df[col].dtype,df[col].isna().sum(),
                          df[col].unique()[:5]) for col in df.columns],
                       columns=['name','nunique','dtype','missing','value :5'])

datainfo(data)

In [ ]:
# income have 'space'

data.columns=[col.strip() for col in data.columns]

# Income punctuation

data['Income']=data['Income'].str.replace(r'[$,\. ]','')

# fill with median & change dtype object to float

data['Income']=data['Income'].fillna(data['Income'].median()).astype(float)

# change to datetime

data['Dt_Customer']=pd.to_datetime(data['Dt_Customer'],format='%m/%d/%y')

#### take eyes on data

Year_Birth - outliers. min birth 1893? it could be. but need little slice.  
wine and meat products - big standard deviation. first watch.  

_reference_

https://www.kaggle.com/jackdaoud/marketing-data/discussion/208329 - data info
https://www.crunchbase.com/organization/ifood - ifood company info

it says Founded Date : May 15, 2011.

In [ ]:
# data.select_dtypes(include='number').describe().iloc[:,:10]

# simple plot
# before

plt.figure()
data['Year_Birth'].plot(kind='box');
plt.show()

# slice

data=data.loc[data.Year_Birth>1920]

# after

plt.figure()
data['Year_Birth'].plot(kind='box');
plt.show()

#### take eyes on data

Cmp_num & response - total campaign. -> with id. Working well campaign.  
num purchases - total purchases. -> Loyal customers.  
Mnt products - total products -> famous products.  

In [ ]:
campaign_cols=['AcceptedCmp3','AcceptedCmp4','AcceptedCmp5',
               'AcceptedCmp1','AcceptedCmp2','Response']

purchases_cols=['NumDealsPurchases','NumWebPurchases',
                'NumCatalogPurchases','NumStorePurchases']

amount_cols=['MntWines','MntFruits','MntMeatProducts','MntFishProducts',
             'MntSweetProducts','MntGoldProds']

# feature engineering

data['total_campaign']=data[campaign_cols].sum(axis=1)
data['total_purchases']=data[purchases_cols].sum(axis=1)
data['total_amount']=data[amount_cols].sum(axis=1)

#### inference 

i think campaign is not effective. one of campaign many.   
and i wonder what kind of campaign effective.

In [ ]:
# no campaign (0) many.

inference=data[['ID','total_campaign','total_purchases','total_amount']]

plt.bar(x=range(6),height=inference['total_campaign'].value_counts());

In [ ]:
# response - last campaign good.
plt.figure()
data.loc[data['total_campaign']==1,['AcceptedCmp3','AcceptedCmp4','AcceptedCmp5',
               'AcceptedCmp1','AcceptedCmp2','Response']].sum(axis=0).sort_values(ascending=False).plot(kind='bar');

# in two, response and campaign3 famous.
plt.figure()
data.loc[data['total_campaign']==2,['AcceptedCmp3','AcceptedCmp4','AcceptedCmp5',
               'AcceptedCmp1','AcceptedCmp2','Response']].sum(axis=0).sort_values(ascending=False).plot(kind='bar');

#### take eyes on data

each campaign high. but almost campaign weak.  

In [ ]:
# apriori - campaign.
from mlxtend.frequent_patterns import apriori

In [ ]:
cmp=data[['AcceptedCmp3','AcceptedCmp4','AcceptedCmp5',
          'AcceptedCmp1','AcceptedCmp2','Response']]

cmp_freq=apriori(cmp,min_support=0.01,use_colnames=True)
cmp_freq.sort_values(by='support',ascending=False)

#### remarkable

loyal customer id : 1991, 7348, 5371, 4073

In [ ]:
purchases=data['total_purchases'].value_counts()
text={}
for idx,v in zip(purchases.index,purchases):
    text[str(data.loc[idx,'ID'])]=v

In [ ]:
from wordcloud import WordCloud

wordcloud=WordCloud(width=3000,height=2000,
                    random_state=71).generate_from_frequencies(text)

plt.imshow(wordcloud,interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# famous products
data[amount_cols].sum(axis=0).sort_values().plot(kind='bar');

# Section 02: Statistical Analysis

### question

* What factors are significantly related to the number of store purchases?
* Does US fare significantly better than the Rest of the World in terms of total purchases?
* Your supervisor insists that people who buy gold are more conservative. Therefore, people who spent an above average amount on gold in the last 2 years would have more in store purchases. Justify or refute this statement using an appropriate statistical test.
* Fish has Omega 3 fatty acids which are good for the brain. Accordingly, do "Married PhD candidates" have a significant relation with amount spent on fish? What other factors are significantly related to amount spent on fish? (Hint: use your knowledge of interaction variables/effects).
* Is there a significant relationship between geographical regional and success of a campaign?

### answer

1. kidhome corr negative. income positive corr. and relative with other product and just because pearsonr, campaign5 affect. 

2. no 

3. store_purchases and gold amouunt columns of p_value. under 0.05 almost 0. and corr score 0.38 weak correlation. relative. 

4. no. weak. rather, others have a lot. in heatmap, relative with other products. campaign1 little corr.

5. SP takes campaign good.

In [ ]:
# question 2-1

num_cols=data.select_dtypes(include='number').columns
cat_cols=[col for col in data.columns if col not in num_cols]

In [ ]:
cat_with_purchases=data[cat_cols+['total_purchases']]
num_with_purchases=data[num_cols]

In [ ]:
# e.g. cat_with_purchases['Education'].value_counts()
# order [201,54,1127,370,485]

edu_pchs=cat_with_purchases.groupby(['Education'],as_index=False)['total_purchases'].sum()
temp=cat_with_purchases['Education'].value_counts()
temp.index=edu_pchs['Education'].tolist()
edu_pchs['total_purchases']=edu_pchs['total_purchases']/temp.tolist()
display(edu_pchs.style.background_gradient(cmap=plt.get_cmap('binary')))

marry_pchs=cat_with_purchases.groupby(['Marital_Status'],as_index=False)['total_purchases'].sum()
temp=cat_with_purchases['Marital_Status'].value_counts()
temp.index=marry_pchs['Marital_Status'].tolist()
marry_pchs['total_purchases']=marry_pchs['total_purchases']/temp.tolist()
display(marry_pchs.style.background_gradient(cmap=plt.get_cmap('binary')))

country_pchs=cat_with_purchases.groupby(['Country'],as_index=False)['total_purchases'].sum()
temp=cat_with_purchases['Country'].value_counts()
temp.index=country_pchs['Country'].tolist()
country_pchs['total_purchases']=country_pchs['total_purchases']/temp.tolist()
display(country_pchs.style.background_gradient(cmap=plt.get_cmap('binary')))

In [ ]:
corr=num_with_purchases.drop(['ID'],axis=1).corr()

fig,ax=plt.subplots(1,1,figsize=(16,16),dpi=200)

sns.heatmap(corr,ax=ax,annot=True,fmt='.2f',square=True)

plt.show()

In [ ]:
# question 2-2

temp=data.groupby('Country')['total_purchases'].sum().sort_values(ascending=False)

fig,ax=plt.subplots(1,1,figsize=(15,8),dpi=200)

ax.bar(x=temp.index,height=temp)

plt.show()

In [ ]:
temp=data.groupby('Country')['total_amount'].sum().sort_values(ascending=False)

fig,ax=plt.subplots(1,1,figsize=(15,8),dpi=200)

ax.bar(x=temp.index,height=temp)

plt.show()

In [ ]:
# question 2-3

# p_value : Probability of extreme results. 
# so, 0.00 says rarely. 

from scipy.stats import pearsonr

corr,p_value=pearsonr(data['MntGoldProds'],data['NumStorePurchases'])
temp=data[['MntGoldProds','NumStorePurchases']].corr()

fig,ax=plt.subplots(1,1,figsize=(15,15),dpi=200)

sns.heatmap(temp,annot=True,ax=ax,square=True)

ax.set_title(f'corr : {corr:.2f} | p_value : {p_value:.2f}')

plt.show()

In [ ]:
# question 2-4.

data['Education_PhD']=data['Education'].apply(lambda x:'1' if x=='PhD' else '0')
data['Marital_Status_Married']=data['Marital_Status'].apply(lambda x:'1' if x=='Married' else '0')

data['Married_PhD']=data['Marital_Status_Married']+data['Education_PhD']

In [ ]:
# 5187 
# i don't think so.

temp=data.groupby(['Married_PhD'],as_index=False)['MntFishProducts'].sum()
display(temp)

fig,ax=plt.subplots(1,1,figsize=(15,10),dpi=200)

sns.boxplot(x='Married_PhD',y='MntFishProducts',data=data,ax=ax)

plt.show()

In [ ]:
# question 2-5.
# SP takes many compaign almost. 
# and all country accepted campaign2 low. 

data.groupby(['Country'])[['AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Response']].sum().style.background_gradient(cmap=plt.get_cmap('Blues'))

# Section 03: Data Visualization

* Which marketing campaign is most successful?
* What does the average customer look like for this company?
* Which products are performing best?
* Which channels are underperforming?

1. response
2. i don't know mean. below i draw plot about average customer look.
3. wines and meats.
4. directly. next web.

In [ ]:
temp=data[campaign_cols].sum(axis=0)

temp.sort_values(ascending=False,inplace=True)

fig,ax=plt.subplots(1,1,figsize=(4,6),dpi=200)

ax.barh(y=temp.index,width=temp,height=0.7)

plt.show()

In [ ]:
def avg_col(df,col):
    cat_cols=['Year_Birth','Kidhome','Teenhome',
              'Country','Education','Marital_Status','AcceptedCmp1',
             'AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5',
             'Response','Complain']
    if col in cat_cols:
        temp=df[col].value_counts().index[0]
    else:
        temp=df[col].median()
    return temp

In [ ]:
# it seems number but category.

data['Year_Birth']=data['Year_Birth'].astype(str)
data['Kidhome']=data['Kidhome'].astype(str)
data['Teenhome']=data['Teenhome'].astype(str)
data['AcceptedCmp1']=data['AcceptedCmp1'].astype(str)
data['AcceptedCmp2']=data['AcceptedCmp2'].astype(str)
data['AcceptedCmp3']=data['AcceptedCmp3'].astype(str)
data['AcceptedCmp4']=data['AcceptedCmp4'].astype(str)
data['AcceptedCmp5']=data['AcceptedCmp5'].astype(str)
data['Response']=data['Response'].astype(str)
data['Complain']=data['Complain'].astype(str)

In [ ]:
data=data.drop(['total_campaign','total_purchases','total_amount',
                'Education_PhD','Marital_Status_Married','Married_PhD'],axis=1)

for col in data.columns:
    if col=='ID':
        continue
    temp=avg_col(data,col)
    
    if isinstance(temp,float):
        print(Fore.WHITE+f"customer's avg {col} : "+Fore.CYAN+f"{temp:.2f}")
    else:
        print(Fore.WHITE+f"customer's most show {col} : "+Fore.CYAN+f"{temp}")

In [ ]:
# percentage

temp=data[amount_cols].sum(axis=0)/1355048
temp=temp.sort_values(ascending=True)

fig,ax=plt.subplots(1,1,figsize=(4,6),dpi=200)

ax.barh(y=temp.index,width=temp)

plt.show()

In [ ]:
# percentage

temp=data[purchases_cols].sum(axis=0)/45165
temp=temp.sort_values(ascending=True)

fig,ax=plt.subplots(1,1,figsize=(4,6),dpi=200)

ax.barh(y=temp.index,width=temp)

plt.show()

# Conclusion

## Suggest

0. please see question and answer. 
1. etc. you should think loyal customer.
2. etc. marketing response good. but the others not. 
3. in store, web visit and web purchases not positive correlation. you should think web marketing. 
4. Recency median value : 49 days. long. you should think conversion.  
5. Intensive marketing for Country SP. SP good for you.